In [ ]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

In [1]:
# %%capture
# %pip install unsloth
# # Also get the latest nightly Unsloth!
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# # Install Flash Attention 2 for softcapping support
# import torch
# if torch.cuda.get_device_capability()[0] >= 8:
#     %pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [2]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "false"

from unsloth import FastLanguageModel
import torch

# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# max_seq_length = 4096  # 공식 튜토리얼은 2048
max_seq_length = 512
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.


model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit", # 공식 튜토리얼
    # model_name="unsloth/gemma-2-9b",  # 모델 이름을 설정합니다.
    model_name="yanolja/EEVE-Korean-10.8B-v1.0",
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
tokenizer

LlamaTokenizerFast(name_or_path='yanolja/EEVE-Korean-10.8B-v1.0', vocab_size=40960, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 2, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
    #                   "gate_proj", "up_proj", "down_proj",],
    target_modules= ["q_proj", "k_proj", "v_proj"],
    lora_alpha = 4,
    lora_dropout = 0.01, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.11.5 patched 48 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# # dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# # dataset = dataset.map(formatting_prompts_func, batched = True,)

In [6]:
prompt_input_template = """아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 입력:
{input}

### 응답:"""


prompt_no_input_template = """아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 응답:"""

def generate_prompt(data_point):
    instruction = data_point["instruction"]
    input = data_point["input"]
    label = data_point["output"]

    if input:
        res = prompt_input_template.format(instruction=instruction, input=input)
    else:
        res = prompt_no_input_template.format(instruction=instruction)

    if label:
        res = f"{res}{label}<|im_end|>" # eos_token을 마지막에 추가

    data_point['text'] = res

    return data_point

In [ ]:
import json
from datasets import Dataset
import pandas as pd

file_path = "data/instruction_dataset_all_rows_all_scenarios_gpt35_tourAPI_tourist_attractions_seoul_gyeonggi_2.4K_scenario.jsonl"

json_data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        json_data.append(json.loads(line.strip()))

for row in json_data:
    for key, value in row.items():
        if isinstance(value, (dict, list)):  # 중첩된 데이터 확인
            row[key] = json.dumps(value)  # 문자열로 변환

df = pd.DataFrame(json_data)

dataset2 = Dataset.from_pandas(df)

def combine_columns(example):
    instruction = example['instruction']
    # input_text = example['input'] if example['input'] else ""  # input이 비어있을 경우를 처리
    input_text = example['input']
    output_text = example['output']

    # 원하는 형식으로 텍스트를 결합
    combined_text = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
                     f"Write a response that appropriately completes the request.\n\n"
                     f"### Instruction:\n{instruction}\n\n"
                     f"### Input:\n{input_text}\n\n"
                     f"### Response:\n{output_text}</s>")

    example["text"] = combined_text
    return example


dataset2 = dataset2.map(combine_columns)
# dataset2 = dataset2.map(formatting_prompts_func, batched = True, )

# remove_column_keys = dataset2.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = dataset2.shuffle().map(generate_prompt)

Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [8]:
dataset_cvted[0]

{'instruction': '방문 시 유의사항 안내',
 'input': '물의정원을 방문할 때 주의사항이 있을까요?',
 'output': '현재 데이터는 방문 유의사항 정보를 제공하지 않습니다.',
 'text': '아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.\n\n### 지시사항:\n방문 시 유의사항 안내\n\n### 입력:\n물의정원을 방문할 때 주의사항이 있을까요?\n\n### 응답:현재 데이터는 방문 유의사항 정보를 제공하지 않습니다.<|im_end|>'}

In [9]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs

# remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True)

Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [10]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [11]:
tokenizer

LlamaTokenizerFast(name_or_path='yanolja/EEVE-Korean-10.8B-v1.0', vocab_size=40960, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

eval_step = 20

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    # train_dataset = dataset2,
    train_dataset = dataset_tokenized,
    data_collator = collate_fn,
    dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    max_seq_length = 512,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs= 3,
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 30,
        # max_steps = 500, # Set num_train_epochs = 1 for full training runs
        learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 20,
        # optim = "adamw_8bit",
        optim = "paged_adamw_8bit",
        save_strategy="steps",
        save_steps=eval_step,
        save_total_limit=2,
        weight_decay = 0.01,
        # lr_scheduler_type = "linear",
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [24]:
import torch
torch.cuda.empty_cache()

In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,543 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 158
 "-____-"     Number of trainable parameters = 1,769,472


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from unsloth import FastLanguageModel
import torch

# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# max_seq_length = 4096  # 공식 튜토리얼은 2048
max_seq_length = 512
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit", # 공식 튜토리얼
    # model_name="unsloth/gemma-2-9b",  # 모델 이름을 설정합니다.
    # model_name="yanolja/EEVE-Korean-10.8B-v1.0",
    model_name="/data/jaesung/yanolja/test_code/outputs/checkpoint-387",
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

/data/jaesung/anaconda3/envs/yanolja/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.527 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.53s/it]
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`
Unsloth 2024.10.7 patched 48 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [1]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    prompt_input_template.format(
        instruction = "특정 지역에서 가볼만 한 곳 추천", # instruction
        input = "서울 종로구에서 가볼만한 관광지를 추천해줘.", # input
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)

response = tokenizer.batch_decode(outputs)

NameError: name 'FastLanguageModel' is not defined

In [ ]:
response[0].split("### 응답:")[1]

"경기도 화성시에서 가볼만한 관광지로는 '화성행궁'을 추천합니다. 화성행궁은 정조가 아버지인 사도세자의 묘를 참배하기 위해 행궁을 지은 곳으로, 화성행궁은 정조의 효심이 담긴 곳입니다. 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴 곳으로, 화성행궁은 정조의 효심이 담긴"

In [47]:
{"instruction": "특정 지역에서 가볼만 한 곳 추천", "input": "서울 중랑구에서 가볼만한 관광지를 추천해주세요.", "output": "서울 중랑구에서 가볼만한 관광지로는 봉화산을 추천합니다. 봉화산은 중랑구 신내동에 위치하고 있으며, 높이는 160.1m로 평지에 돌출된 독립 구릉입니다. 이곳은 서울의 동북부 외곽에 위치하여 북쪽으로는 불암산, 도봉산과 양주 일대를 잘 조망할 수 있습니다. 봉화산은 소나무가 주요 수종으로 자리하고 있으며, 정상 부근에는 산신각이 있어서 음력 3월 3일에는 봉화산 도당제가 열립니다. 또한, 화장실이 있어 편리하게 이용할 수 있습니다. 주변 자연 경관을 감상하며 산책하기에 좋은 관광지입니다."}


{'instruction': '특정 지역에서 가볼만 한 곳 추천',
 'input': '서울 중랑구에서 가볼만한 관광지를 추천해주세요.',
 'output': '서울 중랑구에서 가볼만한 관광지로는 봉화산을 추천합니다. 봉화산은 중랑구 신내동에 위치하고 있으며, 높이는 160.1m로 평지에 돌출된 독립 구릉입니다. 이곳은 서울의 동북부 외곽에 위치하여 북쪽으로는 불암산, 도봉산과 양주 일대를 잘 조망할 수 있습니다. 봉화산은 소나무가 주요 수종으로 자리하고 있으며, 정상 부근에는 산신각이 있어서 음력 3월 3일에는 봉화산 도당제가 열립니다. 또한, 화장실이 있어 편리하게 이용할 수 있습니다. 주변 자연 경관을 감상하며 산책하기에 좋은 관광지입니다.'}